## Descrição
Nesta tarefa você deve ler um arquivo de regiões metropolitanas e equipes esportivas associadas de assets/wikipedia_data.html e responder algumas perguntas sobre cada região metropolitana. Cada uma dessas regiões pode ter um ou mais times do "Big 4": NFL (futebol, em assets/nfl.csv), MLB (beisebol, em assets/mlb.csv), NBA (basquete, em assets/nba. csv ou NHL (hóquei, em assets/nhl.csv) Lembre-se de que todas as perguntas são da perspectiva da região metropolitana e que este arquivo é a "fonte de autoridade" para a localização de um determinado time esportivo. Assim, as equipes que são comumente conhecidas por uma área diferente (por exemplo, "Oakland Raiders") precisam ser mapeadas para a região metropolitana fornecida (por exemplo, área da baía de São Francisco). Isso exigirá alguma compreensão de dados humanos fora dos dados que você recebeu (por exemplo, você terá que codificar manualmente alguns nomes e pode precisar pesquisar no Google para descobrir onde estão as equipes)!

Para cada esporte eu gostaria que você respondesse a pergunta: qual é a correlação da relação vitórias/derrotas com a população da cidade em que está inserido? A proporção de vitórias/derrotas refere-se ao número de vitórias sobre o número de vitórias mais o número de derrotas. Lembre-se que para calcular a correlação com pearsonr, você enviará duas listas ordenadas de valores, as populações do arquivo wikipedia_data.html e a proporção de vitórias/derrotas de um determinado esporte na mesma ordem. Faça a média das taxas de vitórias/derrotas para as cidades que têm várias equipes de um único esporte. Cada esporte vale uma quantia igual nesta tarefa (20%*4=80%) da nota para esta tarefa. Você só deve usar dados do ano de 2018 para sua análise - isso é importante!
## Notas
Não inclua dados sobre o MLS ou CFL em nenhum trabalho que você está fazendo, estamos interessados ​​apenas no Big 4 nesta tarefa.
Eu sugiro que você primeiro aborde as quatro questões de correlação em ordem, pois todas são semelhantes e valem a maioria das notas para esta tarefa. Isso é por design!
É um jogo justo conversar com seus pares sobre estratégia de alto nível, bem como a relação entre áreas metropolitanas e equipes esportivas. No entanto, não poste aspectos de resolução de código da tarefa (incluindo dicionários mapeando áreas para equipes ou regexes que limparão nomes).
Pode haver mais equipes do que o teste de declarações assertivas, lembre-se de recolher várias equipes em uma cidade em um único valor!


# Questão 1
Para esta questão, calcule a correlação da relação ganho/perda com a população da cidade em que está para o NHL usando dados de 2018.

In [50]:
#filtrar os dados para obter as informações apenas de 2018
#descartar colunas desnecessarias de nhl_df
#limpar os nomes do nhl_df a maioria dos nomes tem um * no final deles 
#limpar os nomes de cities alguns nomes tem [note] no final deles
#no nhl os nomes dos times tão misturados com o nome da cidade, preciso separar eles e colocar uma nova coluna somente com o nome dos times
#remover as linhas de divisao 
#existem 4 times com a mesma região vamos agrupar eles e obter o numro de vitorias e derrotas 
#criar a coluna proporção de vitorias 
#mudar o tipo de dado de win e loss para float 
#fazr o population_by_region e win_loss_by_region


In [17]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("/content/nhl.csv")
cities=pd.read_html("/content/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

############### Tratando o cities data Frame ################

#limpar os nomes de cities alguns nomes tem [note] no final deles
def team_name(x):
  return re.sub('\[note \d+\]',repl = '', string = x)
cities['NHL'] = cities['NHL'].apply(lambda x: team_name(x))

#criando um novo dataframe com as variaveis importantes de cities 
new_df= cities[['Metropolitan area', 'NHL','Population (2016 est.)[8]']]
#setando o NHL como index para poder aplicar o drop e remover as cidades que não tem times 
new_df.set_index('NHL', inplace = True)
new_df.drop(['—', ''], axis=0, inplace = True)

###########tratando o nhl_df###############################

nhl_df =nhl_df[nhl_df['year'] == 2018]#pegando dados de 2018
nhl_df.drop([0, 9, 18, 26], axis=0, inplace = True)#remover as linhas de divisao
#limpar os nomes do nhl_df a maioria dos nomes tem um * no final deles 
nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith('*') else x.strip())

#função explicada na chamada com apply
def pegando_regiao(time):
  for x in list(new_df.index.values):
    if time in x:
      return new_df.at[x, 'Metropolitan area']
#este primeiro aplly ira fazer uma coluna que ira conter o nome dos times 
nhl_df['regiao'] = nhl_df['team'].apply(lambda x: x.split(' ')[-1])
#este segundo apply ira comparar a nova coluna que contem o nome dos times com a coluna NHL do new_DF que tbm possui o nome dos times
#sempre que der match ele ira retornar o nome da 'Metropolitan area' substituindo os nomes do time da nova coluna criada anteriormente 
nhl_df['regiao'] = nhl_df['regiao'].apply(lambda x: pegando_regiao(x))

#fazendo um groupby por região para somar os valores dos times de uma mesma região 

agrup = []
for chave, x in nhl_df.groupby('regiao'):
  wins = np.sum(pd.to_numeric(x['W']))
  loses = np.sum(pd.to_numeric(x['L'])) #usando pd.to_numeric para converter string para float
  prop = wins / (wins+loses)
  #adicionando o resultado de cada iteração no dicionario e colocando na lista
  x = {'regiao':chave,'proporcao':prop}
  agrup.append(x)



#criando um novo dataframe para armazenar o agrup que contem a proporção de vitorias por região
new_df2 = pd.DataFrame(agrup)
new_df2.set_index('regiao', inplace = True) 

#criando um dataframe para armazenar a população de cada região 
popula = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
popula = popula.set_index('Metropolitan area')

#juntando os dois dataframes
df = pd.merge (new_df2,popula, how='inner',left_index=True,right_index = True)
df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])

def nhl_correlation(): 
    
        
    population_by_region = list(df['Population (2016 est.)[8]']) # passagem na população da área metropolitana das cidades
    win_loss_by_region = list(df['proporcao']) # passagem na proporção de vitórias/derrotas de nhl_df na mesma ordem das cidades["Metropolitan area"]
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
   
nhl_correlation()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

0.012308996455744264

# Questão 2

Para esta questão, calcule a correlação da relação ganho/perda com a população da cidade em que está para a **NBA** usando dados de **2018**.



In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
nba_df=pd.read_csv("/content/nba.csv")
cities=pd.read_html("/content/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


###############Cities###################

#limpar os nomes de cities alguns nomes tem [note] no final deles
def team_name(x):
  return re.sub('\[note \d+\]',repl = '', string = x)
cities['NBA'] = cities['NBA'].apply(lambda x: team_name(x))

#criando um novo dataframe com as variaveis importantes de cities 
new_df= cities[['Metropolitan area', 'NBA','Population (2016 est.)[8]']]
#setando o NHL como index para poder aplicar o drop e remover as cidades que não tem times 
new_df.set_index('NBA', inplace = True)
new_df.drop(['—', ''], axis=0, inplace = True)


####################NBA##################
nba_df =nba_df[nba_df['year'] == 2018]#pegando dados de 2018

#limpar os nomes do nba_df a maioria dos nomes tem um * e (n) no final deles
def team_name2(x):
  return re.sub(r"(\*)*\s\(\d+\)",repl = '', string = x)
nba_df['team'] = nba_df['team'].apply(lambda x: team_name2(x))

#função explicada na chamada com apply
def pegando_regiao(time):
  for x in list(new_df.index.values):
    if time in x:
      return new_df.at[x, 'Metropolitan area']
#este primeiro aplly ira fazer uma coluna que ira conter o nome dos times 
nba_df['regiao'] = nba_df['team'].apply(lambda x: x.split(' ')[-1])
#este segundo apply ira comparar a nova coluna que contem o nome dos times com a coluna NBA do new_DF que tbm possui o nome dos times
#sempre que der match ele ira retornar o nome da 'Metropolitan area' substituindo os nomes do time da nova coluna criada anteriormente 
nba_df['regiao'] = nba_df['regiao'].apply(lambda x: pegando_regiao(x))


#fazendo um groupby por região para somar os valores dos times de uma mesma região 

agrup = []
for chave, x in nba_df.groupby('regiao'):
  wins = np.sum(pd.to_numeric(x['W']))
  loses = np.sum(pd.to_numeric(x['L'])) #usando pd.to_numeric para converter string para float
  prop = wins / (wins+loses)
  #adicionando o resultado de cada iteração no dicionario e colocando na lista
  x = {'regiao':chave,'proporcao':prop}
  agrup.append(x)


#criando um novo dataframe para armazenar o agrup que contem a proporção de vitorias por região
new_df2 = pd.DataFrame(agrup)
new_df2.set_index('regiao', inplace = True) 

#criando um dataframe para armazenar a população de cada região 
popula = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
popula = popula.set_index('Metropolitan area')

#juntando os dois dataframes
df = pd.merge (new_df2,popula, how='inner',left_index=True,right_index = True)
df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])

def nba_correlation(): 
    
        
    population_by_region = list(df['Population (2016 est.)[8]']) # passagem na população da área metropolitana das cidades
    win_loss_by_region = list(df['proporcao']) # passagem na proporção de vitórias/derrotas de nhl_df na mesma ordem das cidades["Metropolitan area"]
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


nba_correlation()







/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

-0.17657160252844614


#Questão 3
Para esta questão, calcule a correlação da relação ganho/perda com a população da cidade em que está para a MLB usando dados de 2018.

In [26]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
mlb_df=pd.read_csv("/content/mlb.csv")
cities=pd.read_html("/content/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


###############Cities###################

#limpar os nomes de cities alguns nomes tem [note] no final deles
def team_name(x):
  return re.sub('\[note \d+\]',repl = '', string = x)
cities['MLB'] = cities['MLB'].apply(lambda x: team_name(x))

#criando um novo dataframe com as variaveis importantes de cities 
new_df= cities[['Metropolitan area', 'MLB','Population (2016 est.)[8]']]
#setando o NHL como index para poder aplicar o drop e remover as cidades que não tem times 
new_df.set_index('MLB', inplace = True)
new_df.drop(['—', ''], axis=0, inplace = True)

####################MLB##################
mlb_df=mlb_df[mlb_df['year'] == 2018]#pegando dados de 2018

#função explicada na chamada com apply
def pegando_regiao(time):
  for x in list(new_df.index.values):
    if time in x:
      return new_df.at[x, 'Metropolitan area']
#este primeiro aplly ira fazer uma coluna que ira conter o nome dos times 
mlb_df['regiao'] = mlb_df['team'].apply(lambda x: x.split(' ')[-1])
#este segundo apply ira comparar a nova coluna que contem o nome dos times com a coluna MLB do new_DF que tbm possui o nome dos times
#sempre que der match ele ira retornar o nome da 'Metropolitan area' substituindo os nomes do time da nova coluna criada anteriormente 
mlb_df['regiao'] = mlb_df['regiao'].apply(lambda x: pegando_regiao(x))
mlb_df.at[0, 'regiao'] = 'Boston'
#fazendo um groupby por região para somar os valores dos times de uma mesma região 

agrup = []
for chave, x in mlb_df.groupby('regiao'):
  wins = np.sum(pd.to_numeric(x['W']))
  loses = np.sum(pd.to_numeric(x['L'])) #usando pd.to_numeric para converter string para float
  prop = wins / (wins+loses)
  #adicionando o resultado de cada iteração no dicionario e colocando na lista
  x = {'regiao':chave,'proporcao':prop}
  agrup.append(x)


#criando um novo dataframe para armazenar o agrup que contem a proporção de vitorias por região
new_df2 = pd.DataFrame(agrup)
new_df2.set_index('regiao', inplace = True) 

#criando um dataframe para armazenar a população de cada região 
popula = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
popula = popula.set_index('Metropolitan area')

#juntando os dois dataframes
df = pd.merge (new_df2,popula, how='inner',left_index=True,right_index = True)
df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])

def mlb_correlation(): 
    
        
    population_by_region = list(df['Population (2016 est.)[8]']) # passagem na população da área metropolitana das cidades
    win_loss_by_region = list(df['proporcao']) # passagem na proporção de vitórias/derrotas de nhl_df na mesma ordem das cidades["Metropolitan area"]
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


mlb_correlation()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0.1505230448710485

In [22]:
mlb_df

,team,W,L,W-L%,GB,year,League,regiao
0,Boston Red Sox,108,54,0.667,--,2018,MLB,Chicago
1,New York Yankees,100,62,0.617,8.0,2018,MLB,New York City
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB,Tampa Bay Area
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB,Toronto
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB,Baltimore
5,Cleveland Indians,91,71,0.562,--,2018,MLB,Cleveland
6,Minnesota Twins,78,84,0.481,13.0,2018,MLB,Minneapolis–Saint Paul
7,Detroit Tigers,64,98,0.395,27.0,2018,MLB,Detroit
8,Chicago White Sox,62,100,0.383,29.0,2018,MLB,Chicago
9,Kansas City Royals,58,104,0.358,33.0,2018,MLB,Kansas City
